<a href="https://colab.research.google.com/github/Elwing-Chou/acerml/blob/main/poem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from urllib.request import urlretrieve
url = "https://github.com/Elwing-Chou/acerml/raw/main/poem_train.txt"
urlretrieve(url, "train.csv")
url = "https://github.com/Elwing-Chou/acerml/raw/main/poem_test.txt"
urlretrieve(url, "test.csv")

In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding="utf-8")
test_df = pd.read_csv("test.csv", encoding="utf-8")

In [ ]:
test_df

In [ ]:
# 進去前, input, output數字
# unique/value_counts
# Series.replace({"李白":0})
u = train_df["作者"].unique()
writer_index = {name:i for i, name in enumerate(u)}
index_writer = {v:k for k, v in writer_index.items()}
y_train = train_df["作者"].replace(writer_index)
y_test = test_df["作者"].replace(writer_index)
y_test

In [ ]:
import jieba
# example: 回頭檢查換行還有標點有無去掉
p = train_df["內容"][0]
" ".join(jieba.cut(p))
# apply: Series.apply(func名字)
def poemcut(p):
    return " ".join(jieba.cut(p))
x_train = train_df["內容"].apply(poemcut)
x_test = test_df["內容"].apply(poemcut)
x_test

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_count = vec.fit_transform(x_train)
x_test_count = vec.transform(x_test)

In [ ]:
# vec.vocabulary_["。"]
# vec.vocabulary_

In [ ]:
# 詳細:print
x_train_count

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.01)
clf.fit(x_train_count, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_count)
accuracy_score(pre, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(y_test, pre),
             index=[name + "(True)" for name in u],
             columns=[name + "(Pred)" for name in u])

In [ ]:
p = input("輸入詩:")
test = vec.transform([poemcut(p)])
prob = clf.predict_proba(test)[0]
for name, proba in zip(u, prob):
    print(name, "的機率:", proba)